In [13]:
!pip install sympy==1.12


   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   -------------- ------------------------- 2.1/5.7 MB 16.8 MB/s eta 0:00:01
   -------------------------------------- - 5.5/5.7 MB 14.0 MB/s eta 0:00:01
   ---------------------------------------- 5.7/5.7 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.7.0.dev20250113+cu126 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.


In [14]:
import scanpy as sc
import TOSICA
import os
os.environ["PYTORCH_SYMBOLIC_SHAPE_INFERENCE"] = "0"

# Step 1: Load the h_pancreas datasets
# Replace the file paths with the actual paths to your datasets
train_path = "C:/Users/gaiacronus/Downloads/work/combine/adata/hPancreas_train_adata.h5ad"  # Training dataset
test_path = "C:/Users/gaiacronus/Downloads/work/combine/adata/hPancreas_test_adata.h5ad"   # Testing dataset

# Load the datasets as AnnData objects
train_adata = sc.read_h5ad(train_path)
test_adata = sc.read_h5ad(test_path)

# Step 2: Preprocess the datasets
# Align the gene names in both datasets (ensure they have the same genes in the same order)
common_genes = train_adata.var_names.intersection(test_adata.var_names)
train_adata = train_adata[:, common_genes]
test_adata = test_adata[:, common_genes]

# Optional: Normalize the data (if not already normalized)
sc.pp.normalize_total(train_adata, target_sum=1e4)
sc.pp.log1p(train_adata)

sc.pp.normalize_total(test_adata, target_sum=1e4)
sc.pp.log1p(test_adata)
ref_adata=train_adata
query_adata=test_adata
# Step 3: Train the TOSICA model
# Define the label key for cell type annotation in the training dataset
label_key = "Celltype2"  # Replace with the actual key for cell type labels in your data
gmt_path = "C:/Users/gaiacronus/Downloads/work/combine/TOSICA/resources/immune.gmt"  # Path to the gene set file (required by TOSICA)
project_name = "tostry"    # Name of the project directory

# Train the model
TOSICA.train(
    ref_adata,
    gmt_path=gmt_path,
    project=project_name,
    epochs=3,
    label_name=label_key
)

# Step 4: Predict cell types in the test dataset
# Path to the trained model weights (saved in the project directory during training)
model_weight_path = f"{project_name}/model_weight.h5"

# Predict cell types
predicted_adata = TOSICA.pre(
    query_adata,
    model_weight_path=model_weight_path,
    project=project_name
)

# Step 5: Inspect the predictions
# Predicted cell types and probabilities are added to the `obs` of the AnnData object
print(predicted_adata.obs.head())  # Check the predicted labels and probabilities

# Save the results for further analysis
predicted_adata.write("h_pancreas_predicted.h5ad")


C:\Users\gaiacronus\.conda\envs\demo3\Lib\site-packages\scanpy\preprocessing\_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
C:\Users\gaiacronus\.conda\envs\demo3\Lib\site-packages\scanpy\preprocessing\_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


cuda:0


C:\Users\gaiacronus\Downloads\work\combine\TOSICA\train.py:55: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  ct_counts = pd.value_counts(data[:,-1])


Mask loaded!
Model builded!


AttributeError: module 'sympy' has no attribute 'core'